In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as pl

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Minimal class

In [ ]:
class WeibullCDF_IOFunc:
    '''Implements the Weibull CDF function
    f(I) = 1-exp(- ((I-I0)/scale)^k )
    '''
    def __init__(self, I0=0., scale=40., k=10., requires_grad=False, mmax=1., constrained_at_Iref=False, Iref=-20):
        '''
        Args:
            theta: localization parameter (max intensity associated with 0% masking)
            scale: scale parameter (63% masking intensity reached at I0+scale)
            k: shape parameter
            mmax: maximum masking
            constrained_at_Iref: if True, constrains the function to equal 1 at Iref.  (in this case, mmax is superfluous)
            Iref: Iref in dB in the case of 'constrained_at_Iref
        
        '''
        self.I0=I0
        self.scale=scale  
        self.k=k

        self.constrained_at_Iref=constrained_at_Iref
        self._Iref=Iref
        
        self.mmax=mmax


    def __call__(self, I):
        Delta_I=np.maximum((I-self.I0), 0.)
        if self.constrained_at_Iref:
            Delta_I_ref=np.maximum((self._Iref-self.I0), 0.)
            return (1-np.exp( -(Delta_I/self.scale)**self.k))/(1-np.exp( -(Delta_I_ref/self.scale)**self.k))
        else:
            return self.mmax*(1-np.exp( -(Delta_I/self.scale)**self.k))


In [ ]:

Iref = float(100 +10 - 32)
fs=48828
#Iref = 10*np.log10( 10**(Iref/10)/(fs/2) ) #spectral density
#Iref-=20
#Iref=21
Iref=14.2  #1-22

In [ ]:
#NB: remove 'manual' for continuous update
@interact(I0=widgets.FloatSlider(value=-20, min=-40, max=0, step=1),
    scale=widgets.FloatSlider(value=30, min=1, max=100, step=1), 
          k=widgets.FloatSlider(value=10, min=1, max=20, step=0.5))
def plotwbcdf(I0, scale, k):

    wb_cdf=WeibullCDF_IOFunc(I0=I0,
        scale=scale,
        k=k,
        requires_grad=False,
        constrained_at_Iref=True,
        Iref=Iref)
    
    
    wb_cdf2=WeibullCDF_IOFunc(I0=-20.,  #-15
        scale=30.,
        k=5.,
        mmax=1.,
        requires_grad=False,
        constrained_at_Iref=True,
        Iref=Iref)

    I=np.linspace(-10, 30)
    pl.figure()
    
    pl.plot(I, wb_cdf(I))
    
    pl.plot(I, wb_cdf2(I))
    pl.xlim([-10, 30])
    pl.title('Masking IO Function')
    pl.xlabel('Power spectral density (dB)')
    pl.show()

Sigmoid

In [ ]:
#NB: remove 'manual' for continuous update
@interact(mu=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
    a=widgets.FloatSlider(value=0.25, min=0.01, max=2, step=0.1))
def plotsigm(mu, a):

    def sigm(I):
        return 1/(1+np.exp(- a*(I-mu)))
    I=np.linspace(-30, 20)
    pl.figure()
    
    pl.plot(I, sigm(I))
    
    pl.xlim([-20, 20])
    pl.title('Masking IO Function')
    pl.xlabel('Power spectral density (dB)')
    pl.show()

In [ ]:
Delta_I_ref